# GRPC Inference

### Setup

In [ ]:
!pip install grpcio grpcio-tools opencv-python-headless

### Inspecting the gRPC Endpoint

Let's check out the gRPC endpoint's model metadata.

In [ ]:
grpc_host = 'modelmesh-serving'
grpc_port = 8033
model_name = 'yolo'

# Confidence threshold, between 0 and 1 (detections with less score won't be retained)
conf = 0.2

# Intersection over Union Threshold, between 0 and 1 (cleanup overlapping boxes)
iou = 0.6

image_path = 'images/zidane.jpg'


In [ ]:
import grpc
import grpc_predict_v2_pb2
import grpc_predict_v2_pb2_grpc


options = [('grpc.max_receive_message_length', 100 * 1024 * 1024)]
channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}", options=options)
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
response

### Image Preprocessing Functions

First, we need to preprocess and format the image.

In [ ]:
import numpy as np
import cv2

def letterbox(im, color=(114, 114, 114), auto=True, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    new_shape= 640
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)

    # Compute padding
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding

    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return im, r, (dw, dh)


def preprocess(image):
    image = image.transpose((2, 0, 1))  # HWC->CHW for PyTorch model
    image = np.expand_dims(image, 0)  # Model expects an array of images
    image = np.ascontiguousarray(image)  # Speed up things by rewriting the array contiguously in memory
    im = image.astype(np.float32)  # Model expects float32 data type
    im /= 255  # Convert RGB values [0-255] to [0-1]
    return im


def getImage(path, size):
    return cv2.imread(path)



In [ ]:
import time

start_time = time.time()
image_or = getImage(image_path, 640)
letterboxed_image, ratio, dwdh = letterbox(image_or, auto=False)
img_data = preprocess(letterboxed_image)

### Results filtering and transformation utilities

In [ ]:
import classes


def xywh2xyxy(xywh):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    xyxy = np.copy(xywh)
    xyxy[..., 0] = xywh[..., 0] - xywh[..., 2] / 2  # top left x
    xyxy[..., 1] = xywh[..., 1] - xywh[..., 3] / 2  # top left y
    xyxy[..., 2] = xywh[..., 0] + xywh[..., 2] / 2  # bottom right x
    xyxy[..., 3] = xywh[..., 1] + xywh[..., 3] / 2  # bottom right y
    return xyxy

def get_overlapping_box(new_box, existing_boxes, iou_threshhold):
    overlapping_box_index = -1
    for i, existing_box in enumerate(existing_boxes):
        overlap_x_min = np.maximum(new_box['box']['xMin'], existing_box['box']['xMin'])
        overlap_y_min = np.maximum(new_box['box']['yMin'], existing_box['box']['yMin'])
        overlap_x_max = np.minimum(new_box['box']['xMax'], existing_box['box']['xMax'])
        overlap_y_max = np.minimum(new_box['box']['yMax'], existing_box['box']['yMax'])

        # Find out the width and the height of the intersection box
        w = np.maximum(0, overlap_x_max - overlap_x_min + 1)
        h = np.maximum(0, overlap_y_max - overlap_y_min + 1)
        overlap_area = (w * h)

        # compute the ratio of overlap
        max_area = np.maximum(new_box['box']['area'], existing_box['box']['area'])
        overlap = overlap_area / max_area

        # if the actual boungding box has an overlap bigger than overlapThresh
        if overlap > iou_threshhold:
            overlapping_box_index = i
            break
    return overlapping_box_index


def detection_arr2dict(item):
#   item: [middle_X, middle_Y, width, height, obj_confidence, ...class confidences]
    class_index = np.argmax(item[5:])
    xyxy = xywh2xyxy(item[:4])
    return {
        'box': {
            'xMin': xyxy[0],
            'yMin': xyxy[1],
            'xMax': xyxy[2],
            'yMax': xyxy[3],
            'area': item[2] * item[3]
        },
        'classIndex': class_index,
        'class': classes.coco_classes[class_index],
        'label': classes.coco_classes[class_index],
        'overallConfidence': item[4],
        'classConfidence': item[class_index + 5],
        'score': item[4] * item[class_index + 5],
    }


def map_results(detections, overlap_threshhold=0.6, conf_threshhold=0.2, max_detections=20):
    results = []
    # num_detections = min(len(detections), max_detections)

    for i in range(0, len(detections)):        
        if len(results) >= max_detections:
            break

        d = detection_arr2dict(detections[i])
        if d['overallConfidence'] < conf_threshhold:
            break
            
        if d['score'] < conf_threshhold:
            continue
            
        overlapping_box_index = get_overlapping_box(d, results, overlap_threshhold)
        # only append if there's no overlapping box
        if overlapping_box_index < 0:
            results.append(d)
        # replace if the new box has higher confidence
        elif d['score'] > results[overlapping_box_index]['score']:
            results[overlapping_box_index] = d

    return results

### Request Function

Builds and submits our gRPC request.

In [ ]:
import time
import classes


def transform_filter_results(result_arr):
    prediction_columns_number = 5 + len(classes.coco_classes)  # Model returns model returns [xywh, conf, class0, class1, ...]
    reshaped_result_arr = result_arr.reshape(1, int(int(result_arr.shape[0])/prediction_columns_number), prediction_columns_number)
    sorted_result_arr = (reshaped_result_arr[0][reshaped_result_arr[0][:, 4].argsort()])[::-1]
    return map_results(sorted_result_arr)


def rhods_grpc_request(img_data):
    # request content building
    inputs = []
    inputs.append(grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor())
    inputs[0].name = "images"
    inputs[0].datatype = "FP32"
    inputs[0].shape.extend([1, 3, 640, 640])
    arr = img_data.flatten()
    inputs[0].contents.fp32_contents.extend(arr)

    # request building
    request = grpc_predict_v2_pb2.ModelInferRequest()
    request.model_name = model_name
    request.inputs.extend(inputs)

    t1 = time.time()
    response = stub.ModelInfer(request)
    t2 = time.time()
    inference_time = t2-t1
    
    result_arr = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)
    return transform_filter_results(result_arr)

### Run the Request

In [ ]:
# Confidence threshold, between 0 and 1 (detections with less score won't be retained)
conf = 0.2
# Intersection over Union Threshold, between 0 and 1 (cleanup overlapping boxes)
iou = 0.6

image_path = 'images/zidane.jpg'

image_or = getImage(image_path, 640)
letterboxed_image, ratio, dwdh = letterbox(image_or, auto=False)
img_data = preprocess(letterboxed_image)
results = rhods_grpc_request(img_data)
results

### Show the Results

In [ ]:
import classes
import random

def draw_result(img, ratio, dwdh, detections):
    names = classes.coco_classes
    colors = {name:[random.randint(0, 255) for _ in range(3)] for i,name in enumerate(names)}   
    for i,(d) in enumerate(detections):
        box = np.array([d['box']['xMin'], d['box']['yMin'], d['box']['xMax'], d['box']['yMax']])
        box -= np.array(dwdh*2)
        box /= ratio
        box = box.round().astype(np.int32).tolist()
        cls_id = int(d['classIndex'])
        score = round(float(d['score']),3)
        name = names[cls_id]
        color = colors[name]
        name += ' '+str(score)
        cv2.rectangle(img,box[:2],box[2:],color,2)
        cv2.putText(img,name,(box[0], box[1] - 2),cv2.FONT_HERSHEY_SIMPLEX,0.75,[0, 255, 0],thickness=2) 
    return img


In [ ]:
result_image = draw_result(image_or, ratio, dwdh, results)  # Draw the boxes from results

from matplotlib import pyplot as plt
fig = plt.gcf()
fig.set_size_inches(24, 12)
plt.axis('off')
plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
